In [1]:
import os
from glob import glob
import random

import torch
import transformers
from transformers import PreTrainedTokenizerFast
from transformers import AutoTokenizer
from transformers import set_seed

import pynvml

def check_gpu_memory():
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(torch.cuda.current_device())
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    pynvml.nvmlShutdown()

    return info.free / (1024**2)  # Returns free memory in MB

if torch.cuda.is_available():
    print("CUDA (GPU support) is available and PyTorch can use GPUs!")
    device = torch.device("cuda")
else:
    print("CUDA (GPU support) is not available and PyTorch will use the CPU.")
    device = torch.device("cpu")
    raise ValueError("No GPU found, use GGML!")

model_name = "Nethermind/Mpt-Instruct-DotNet-S"
config = transformers.AutoConfig.from_pretrained(out_name, trust_remote_code=True)
config.init_device = device
config.max_seq_len = 1024 
config.attn_config['attn_impl'] = 'torch'
config.use_cache = False


free_memory_MB = check_gpu_memory()
if free_memory_MB >= 15000:
    print("We will run model in the fastest possible configuration.")
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        config=config,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        ignore_mismatched_sizes=True
    )
elif free_memory_MB >= 8000:
    print("We will run model with minimal GPU memory footprint as possible.")
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        config=config,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        ignore_mismatched_sizes=True,
        load_in_8bit=True
    )
else:
    raise ValueError("Insufficient GPU memory!")

model = model.eval()

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.pad_token = tokenizer.eos_token

CUDA (GPU support) is available and PyTorch can use GPUs!
Instantiating an MPTForCausalLM model from /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/925e0d80e50e77aaddaf9c3ced41ca4ea23a1025/modeling_mpt.py
You are using config.init_device=device(type='cuda'), but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


In [10]:
from markdownify import markdownify as md
import ftfy
from IPython.display import display, Markdown

INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
PROMPT_FOR_GENERATION_FORMAT = """{system}
{instruction_key}
{instruction}
{response_key}
""".format(
    system="{system}",
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY
)


def give_answer(instruction="Create a loop over [0, 6, 7 , 77] that prints its contentrs", system="You are an experienced .Net C# developer. Below is an instruction that describes a task. Write a response that completes the request providing detailed explanations with code examples.", ):
    question = PROMPT_FOR_GENERATION_FORMAT.format(system=system, instruction=instruction)
    input_tokens = tokenizer.encode(question ,return_tensors='pt')               
    model.generate(input_tokens.to(device), max_new_tokens=min(512, 1024 - input_tokens.shape[1]), do_sample=False, top_k=1, top_p=0.95)
    outputs = output_loop(tokenized_question)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(answer[0])

    # Convert HTML to Markdown
    markdown = md(ftfy.fix_text(answer[0]))
    display(Markdown(markdown+ "\n----"))
    return markdown

In [11]:
%%time
give_answer("What is the main difference between a struct and a class in C#?")

print("----------------------")

give_answer("""interface IRobot {
    void Take(string what);
    void Cut(int size);
    void Give(string to);
}

You are a robot. Create a sample of _api usage performing what is asked in Example:
Rick asks a robot: slice and pass me the butter, please

Create example using _api continuing this code:
class Request {
    IRobot _api;
    void Do() {
""")


58 1942
You are an experienced.Net C# developer. Below is an instruction that describes a task. Write a response that completes the request providing detailed explanations with code examples.
### Instruction:
What is the main difference between a struct and a class in C#?
### Response:
A struct is a value type that is used to encapsulate small groups of related variables. It is similar to a record in C# 9.0. A class, on the other hand, is a reference type that is used to encapsulate objects with different properties and methods. It is more flexible than a struct and can be used to create objects with properties and methods specific to the class.



You are an experienced.Net C# developer. Below is an instruction that describes a task. Write a response that completes the request providing detailed explanations with code examples.
### Instruction:
What is the main difference between a struct and a class in C#?
### Response:
A struct is a value type that is used to encapsulate small groups of related variables. It is similar to a record in C# 9.0. A class, on the other hand, is a reference type that is used to encapsulate objects with different properties and methods. It is more flexible than a struct and can be used to create objects with properties and methods specific to the class.

----

----------------------
139 1861
You are an experienced.Net C# developer. Below is an instruction that describes a task. Write a response that completes the request providing detailed explanations with code examples.
### Instruction:
interface IRobot {
    void Take(string what);
    void Cut(int size);
    void Give(string to);
}

You are a robot. Create a sample of _api usage performing what is asked in Example:
Rick asks a robot: slice and pass me the butter, please

Create example using _api continuing this code:
class Request {
    IRobot _api;
    void Do() {

### Response:
// Interface declaration
interface IRobot {
    void Take(string what);
    void Cut(int size);
    void Give(string to);
}

// Implementation of the interface
class Robot : IRobot {
    public void Take(string what) {
        // Implementation code
    }

    public void Cut(int size) {
        // Implementation code
    }

    public void Give(string to) {
        // Implementation code
    }
}

// Usage of t

You are an experienced.Net C# developer. Below is an instruction that describes a task. Write a response that completes the request providing detailed explanations with code examples.
### Instruction:
interface IRobot {
 void Take(string what);
 void Cut(int size);
 void Give(string to);
}

You are a robot. Create a sample of \_api usage performing what is asked in Example:
Rick asks a robot: slice and pass me the butter, please

Create example using \_api continuing this code:
class Request {
 IRobot \_api;
 void Do() {

### Response:
// Interface declaration
interface IRobot {
 void Take(string what);
 void Cut(int size);
 void Give(string to);
}

// Implementation of the interface
class Robot : IRobot {
 public void Take(string what) {
 // Implementation code
 }

 public void Cut(int size) {
 // Implementation code
 }

 public void Give(string to) {
 // Implementation code
 }
}

// Usage of the interface
class Program {
 static void Main(string[] args) {
 // Creating an instance of the robot class
 IRobot api = new Robot();

 // Invoking the methods on the interface
 api.Take("Butter");
 api.Cut(5);
 api.Give("To Rick");
 }
}

----

CPU times: user 1min 25s, sys: 480 ms, total: 1min 26s
Wall time: 1min 26s


'You are an experienced.Net C# developer. Below is an instruction that describes a task. Write a response that completes the request providing detailed explanations with code examples.\n### Instruction:\ninterface IRobot {\n void Take(string what);\n void Cut(int size);\n void Give(string to);\n}\n\nYou are a robot. Create a sample of \\_api usage performing what is asked in Example:\nRick asks a robot: slice and pass me the butter, please\n\nCreate example using \\_api continuing this code:\nclass Request {\n IRobot \\_api;\n void Do() {\n\n### Response:\n// Interface declaration\ninterface IRobot {\n void Take(string what);\n void Cut(int size);\n void Give(string to);\n}\n\n// Implementation of the interface\nclass Robot : IRobot {\n public void Take(string what) {\n // Implementation code\n }\n\n public void Cut(int size) {\n // Implementation code\n }\n\n public void Give(string to) {\n // Implementation code\n }\n}\n\n// Usage of the interface\nclass Program {\n static void Main(

In [4]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Aug 20 22:33:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.06              Driver Version: 536.40       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA 